### Week 7 C1 In Class IP Logical Constraints New Flavors Model

### Instructions: Complete the problem and turn in the associated html file with your name displayed at the top. 

## 1. Problem Statement

In this example we have 8 Ice Cream Flavors we are exploring investing in. We have
the costs and prices (in 000s) expected over the next month with each flavor. In addition, we
must follow these constraints. We want to maximize profit.

* Only 1 of each flavor
* Stay within budget of 35
* F8 and F4 mutually exclusive
* Requires 1 Chocolate Flavor (F6/F7)
* F2 is contingent on F5

## 2. Data

In [1]:
import pandas as pd
import pyomo.environ as pe
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
### Creating "nicer" formatted markdown with printmd()
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [3]:
# Read in the data first. 
raw_data = pd.read_excel('w07-c01-logical-in-class.xlsx', sheet_name='New Flavors Model')
raw_data

,Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,ALL CONSTRAINTS TO SATISFY
0,NaN,Flavor,1,2,3.0,4.0,5.0,6.0,7.0,8.0,NaN,In this example we have 8 Ice Cream Flavors we...,NaN,NaN
1,NaN,Cost,9.8,6.4,7.7,9.2,8.3,10.1,9.1,7.4,NaN,the costs and profits (in 000s) expected over ...,NaN,NaN
2,NaN,Price,14.5,12.5,15.6,16.5,14.1,13.4,13.8,14.1,NaN,must follow these constraints.,NaN,NaN
3,Objective: Maximize Revenue,NaN,Price,Cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,•Only 1 of each flavor,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,•Stay within budget of 35,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,•F8 and F4 mutually exclusive,NaN,NaN
6,NaN,NaN,1,2,3.0,4.0,5.0,6.0,7.0,8.0,NaN,•Requires 1 Chocolate Flavor (F6/F7),NaN,NaN
7,Decisions: Accept or not?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,•F2 is contingent on F5,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Constraints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHS,Eq/InEq,RHS,NaN


Create a DataFrame for the profit and costs. View the DataFrame.

In [4]:
DV_indexes = range(1,9)
coef = raw_data.iloc[[1,2],[2,3,4,5,6,7,8,9]]
coef.index=['cost','price']
coef.columns=DV_indexes
coef

,1,2,3,4,5,6,7,8
cost,9.8,6.4,7.7,9.2,8.3,10.1,9.1,7.4
price,14.5,12.5,15.6,16.5,14.1,13.4,13.8,14.1


## 3. Model Definition

In [5]:
model = pe.ConcreteModel()

### Define Decision Variables

In [6]:
# Decision variables
model.x = pe.Var(DV_indexes, domain=pe.Binary)

### Define Objective function

In [7]:
#maximize profit = price - cost
model.obj = pe.Objective(expr=sum([coef.loc['price', c]*model.x[c] 
                                   for c in DV_indexes])
                         - sum([coef.loc['cost',c]*model.x[c] 
                                for c in DV_indexes]),
                         sense=pe.maximize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 14.5*x[1] + 12.5*x[2] + 15.6*x[3] + 16.5*x[4] + 14.1*x[5] + 13.4*x[6] + 13.8*x[7] + 14.1*x[8] - (9.8*x[1] + 6.4*x[2] + 7.7*x[3] + 9.2*x[4] + 8.3*x[5] + 10.1*x[6] + 9.1*x[7] + 7.4*x[8])


### Define Constraints

In [8]:
# Constraints
model.cons_budget = pe.Constraint(expr=sum([coef.loc['cost', c]*model.x[c] 
                                            for c in DV_indexes]) <=35)
model.cons_F4F8ME = pe.Constraint(expr=model.x[4]+model.x[8] <=1)
model.cons_F6F7Choc = pe.Constraint(expr=model.x[6]+model.x[7] >=1)
model.cons_F2ContF5 = pe.Constraint(expr=model.x[5]-model.x[2] >=0)

model.pprint() #prints all objects (obj function, constraints, etc.) defined in the model

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {1, 2, 3, 4, 5, 6, 7, 8}

1 Var Declarations
    x : Size=8, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 14.5*x[1] + 12.5*x[2] + 15.6*x[3] + 16.5*x[4] + 14.1*x[5] + 13.4*x[6] 

## 4. Model Solution

In [9]:
opt = pe.SolverFactory('glpk')
#opt.solve(model,tee=True) 
success=opt.solve(model)
print(success.solver.status,success.solver.termination_condition)

ok optimal


Ok, so we now have our **tactical information**. We know the solution:

### Optimal Objective Value

In [10]:
obj_val = model.obj.expr()
printmd(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $25.70

### Optimal Decision Variables

In [11]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = DV[c].value
DV_solution

,1,2,3,4,5,6,7,8
x,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [12]:
DV_solution.index = ['original']
DV_solution

,1,2,3,4,5,6,7,8
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


Let's create a `results` dataframe that adds on the objective function optimal value in the data frame.

In [13]:
results = DV_solution
results['opt profit'] = obj_val #adds a new column
results

,1,2,3,4,5,6,7,8,opt profit
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7


## 5. Sensitivity Analysis

Now, let's try to get some strategic information to see what else we can learn about this model.

The above model is based on current conditions, but we are wondering how the maximum revenue and choice of decision variables will be affected by potential future changes to our operations. Let's say that we want to test the sensitivity of this model based on 3 different items:
* Increase the price by .50 for flavors 3 or 4.
* Increase the cost of flavor 3 by 1.00
* Increase the cost of flavor 2 by .50 and increase the price by 1.00

### Create a function that runs the model

Copy all the above model creation code without any thing "extraneous" and put it in the function `def run_model()`. At the end use `return model`. Make sure all the code is indented the same under the `def` line.

In [14]:
def run_model():
    model = pe.ConcreteModel()
    # Define Decision Variables
    model.x = pe.Var(DV_indexes, domain=pe.Binary)
    # Define Objective Function for profit
    model.obj = pe.Objective(expr=sum([coef.loc['price', c]*model.x[c] 
                                       for c in DV_indexes])
                             - sum([coef.loc['cost',c]*model.x[c] 
                                    for c in DV_indexes]),
                             sense=pe.maximize)
    #Define Constraints
    model.cons_budget = pe.Constraint(expr=sum([coef.loc['cost', c]*model.x[c] 
                                                for c in DV_indexes]) <=35)
    model.cons_F4F8ME = pe.Constraint(expr=model.x[4]+model.x[8] <=1)
    model.cons_F6F7Choc = pe.Constraint(expr=model.x[6]+model.x[7] >=1)
    model.cons_F2ContF5 = pe.Constraint(expr=model.x[5]-model.x[2] >=0)
    opt = pe.SolverFactory('glpk')
    #opt.solve(model,tee=True) 
    success=opt.solve(model)
    return model

Capture the original `coef` table so we can reset the model each time. Note if we are going to change the values, rather than use a reference we want to use the `.copy()` function so that it will make a hard copy and not just a soft copy and remove references to the original dataframe.

In [15]:
coef_orig = coef.copy()
coef_orig

,1,2,3,4,5,6,7,8
cost,9.8,6.4,7.7,9.2,8.3,10.1,9.1,7.4
price,14.5,12.5,15.6,16.5,14.1,13.4,13.8,14.1


### 5.1 Increase the price by .50 for flavors 3 or 4.

First we'll update the data with price of 15.60+.50 = 16.10 for flavor 3. Change the `coef` table price for flavor 3 to be this new value.

In [16]:
coef = coef_orig.copy() #reset to original
coef.loc['price',3] = coef.loc['price',3] + .5
coef

,1,2,3,4,5,6,7,8
cost,9.8,6.4,7.7,9.2,8.3,10.1,9.1,7.4
price,14.5,12.5,16.1,16.5,14.1,13.4,13.8,14.1


Now rerun the model with your new function and view the new optimal values

In [17]:
model = run_model()

obj_val = model.obj.expr()
printmd(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $26.20

In [18]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = DV[c].value
DV_solution.index = ['Flav 3 price .50']
DV_solution

,1,2,3,4,5,6,7,8
Flav 3 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [19]:
results = results.append(DV_solution)
results.loc['Flav 3 price .50','opt profit'] = obj_val
results

,1,2,3,4,5,6,7,8,opt profit
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7
Flav 3 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2


In [20]:
printmd('Our model if we increase the price of flavor 3 is the same as the original decision variables with higher revenue:')

Our model if we increase the price of flavor 3 is the same as the original decision variables with higher revenue:

Now we'll update the data with price of 16.50+.50 = 17.00 for flavor 4.

In [21]:
#Make sure not to run this more than once - check 17
coef = coef_orig.copy() #reset to original
coef.loc['price',4] = coef.loc['price',4] + .5
coef

,1,2,3,4,5,6,7,8
cost,9.8,6.4,7.7,9.2,8.3,10.1,9.1,7.4
price,14.5,12.5,15.6,17.0,14.1,13.4,13.8,14.1


Then we'll update the model

In [22]:
model = run_model()

obj_val = model.obj.expr()
printmd(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $26.20

In [23]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = DV[c].value
DV_solution.index = ['Flav 4 price .50']
DV_solution

,1,2,3,4,5,6,7,8
Flav 4 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [24]:
results = results.append(DV_solution)
results.loc['Flav 4 price .50','opt profit'] = obj_val
results

,1,2,3,4,5,6,7,8,opt profit
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7
Flav 3 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2
Flav 4 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2


In [25]:
printmd('Our model if we increase the price of flavor 4 is the same as the original decision variables with higher revenue (but same if we increase price of flavor 3.):')

Our model if we increase the price of flavor 4 is the same as the original decision variables with higher revenue (but same if we increase price of flavor 3.):

So based on this sensitivity if we increase the price by .50 for either flavor 3 or 4 we increase revenue by .50 from 25.7 to 26.2.

### 5.2 Increase the cost of flavor 3 by 1.00

Ok, so what if we have to increase costs of flavor 3?

First we'll update the data with cost of flavor 3 of 7.70 +.50 = 8.20.

In [26]:
coef = coef_orig.copy() #reset to original
coef.loc['cost',3] = coef.loc['cost',3] + 1
coef

,1,2,3,4,5,6,7,8
cost,9.8,6.4,8.7,9.2,8.3,10.1,9.1,7.4
price,14.5,12.5,15.6,16.5,14.1,13.4,13.8,14.1


Then we'll update the model

In [27]:
model = run_model()

obj_val = model.obj.expr()
printmd(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $24.10

In [28]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = DV[c].value
DV_solution.index = ['Flav 3 cost 1.00']
DV_solution

,1,2,3,4,5,6,7,8
Flav 3 cost 1.00,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


In [29]:
results = results.append(DV_solution)
results.loc['Flav 3 cost 1.00','opt profit'] = obj_val
results

,1,2,3,4,5,6,7,8,opt profit
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7
Flav 3 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2
Flav 4 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2
Flav 3 cost 1.00,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,24.1


In [30]:
printmd('Our model if we increase the cost of flavor 3 does change the optimal solution and lower revenue from 25.7:')
printmd(f'The maximum revenue is {round(obj_val,2)} and we should invest in flavors 3, 5, 7, and 8.')

Our model if we increase the cost of flavor 3 does change the optimal solution and lower revenue from 25.7:

The maximum revenue is 24.1 and we should invest in flavors 3, 5, 7, and 8.

### 5.3 Increase the cost of flavor 2 by .50 and increase the price by 1.00.

Finally, let's try two changes at once. What if we have to increase the cost of flavor 2 by .50 and the price by 1.00?

First we'll update the data table with the new cost and price for flavor 5.

In [31]:
coef = coef_orig.copy() #reset to original
coef.loc['cost',2] = coef.loc['cost',2] + .5
coef.loc['price',2] = coef.loc['price',2] + 1.0
coef

,1,2,3,4,5,6,7,8
cost,9.8,6.9,7.7,9.2,8.3,10.1,9.1,7.4
price,14.5,13.5,15.6,16.5,14.1,13.4,13.8,14.1


Then we'll update the model

In [32]:
model = run_model()

obj_val = model.obj.expr()
printmd(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $25.70

In [33]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = DV[c].value
DV_solution.index = ['Flav 2 cost .50 price 1.00']
DV_solution

,1,2,3,4,5,6,7,8
Flav 2 cost .50 price 1.00,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [34]:
results = results.append(DV_solution)
results.loc['Flav 2 cost .50 price 1.00','opt profit'] = obj_val
results

,1,2,3,4,5,6,7,8,opt profit
original,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7
Flav 3 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2
Flav 4 price .50,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,26.2
Flav 3 cost 1.00,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,24.1
Flav 2 cost .50 price 1.00,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,25.7


In [35]:
printmd('Our model if we increase the cost and price of flavor 2 it does change the optimal solution and lower revenue from 25.7:')
printmd(f'The maximum revenue is {round(obj_val,2)} and we should invest in flavors 2, 4, 5, and 7.')

Our model if we increase the cost and price of flavor 2 it does change the optimal solution and lower revenue from 25.7:

The maximum revenue is 25.7 and we should invest in flavors 2, 4, 5, and 7.

### Export Notebook As an HTML file to submit on canvas.